## A demo using Hydrogen Hamiltonian with GPT-QE.

In [1]:
import torch
from gqe.mingpt.utils import set_seed


In [2]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, do_generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.util import get_device
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor

# molecule = generate_molecule("Li", "H", 1.596, "sto-3g", bravyi_kitaev=False)
bond_length = 3.0
geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {bond_length}\n" + f"H 0.0 0.0 {2 * bond_length}\n"
molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
nqubit = 12

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIIIIIIIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=4)
pool = DefaultOperatorPool(paulis)
cost = EnergyCost(hamiltonian, initializer, pool,
                  [1 / 320, -1 / 320, 1 / 160, -1 / 160, 1 / 80, -1 / 80, 1 / 40, -1 / 40, 0.05, -0.05, 0.1, -0.1, 0.2, -0.2])


converged SCF energy = -15.0242100060364
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
paulis [+IIIYZXIIIIII, +IIIYZZZXIIII, +IIIYZZZZZXII, +IIIYZZZZZZZX, +IIXXIIIIIIYX, +IIXXIIIIYXII, +IIXXIIYXIIII, +IIXXYXIIIIII,

## FCI energy by diagonalization

In [3]:
from qwrapper.hamiltonian import compute_ground_state

print(compute_ground_state(hamiltonian))

-15.336459383306721


In [4]:
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(12, [], "qulacs")))

hf state: -15.024210006036476


## Setup for GPT

In [5]:
# create a GPT instance
from gqe.mingpt.model import GPT
from gqe.mingpt.trainer import Trainer


def run(seed):
    set_seed(seed)
    model_config = GPT.get_default_config()
    model_config.model_type = 'gpt2'
    model_config.vocab_size = cost.vocab_size()
    model_config.n_gates = 30  # The number of gates for each circuit
    model_config.block_size = model_config.n_gates
    model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
    model_config.embd_pdrop = 0.1
    model_config.resid_pdrop = 0.1
    model_config.attn_pdrop = 0.1
    model_config.std = 0.02
    model_config.energy_offset = 14
    model = GPT(model_config, cost)

    train_config = Trainer.get_default_config()
    train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
    train_config.max_iters = 500
    train_config.num_workers = 10
    train_config.n_samples = 50
    trainer = Trainer(train_config, model)
    file_monitor = FileMonitor()
    callback_generator = DefaultCallback(model, monitors=[PrintMonitor(), file_monitor], del_temperature=0.01)
    trainer.set_callback('on_batch_end', callback_generator.generate())
    trainer.run()
    torch.save(model.state_dict(), '../saved_models/gptqe_test_2')
    file_monitor.save(f'../output/trajectory_beh2_{seed}.json')

In [6]:
for seed in [31, 37, 43, 47, 53]:
    run(seed)

number of parameters: 85.91M
running on device mps
iter_dt 0.00s; iter 0: train loss 0.61095 temperature: 5
mean_logits tensor([-14.8405, -15.1819, -15.2120, -14.9140, -14.9605, -14.7375, -15.2389,
        -14.7798, -14.9196, -15.0656, -15.0907, -15.0688, -14.8682, -15.2256,
        -15.0942, -15.0855, -14.8595, -14.9248, -15.1134, -15.0770, -14.9645,
        -14.9948, -15.0034, -14.8459, -14.9655, -15.0618, -14.9039, -14.7230,
        -15.1477, -15.0719, -14.8178, -15.2054, -15.0087, -15.2419, -14.9690,
        -14.9001, -15.0520, -15.1607, -15.1998, -15.0896, -15.1262, -14.8707,
        -15.2153, -15.0882, -15.1118, -14.9701, -15.1348, -14.8401, -15.2355,
        -15.0764], device='mps:0', grad_fn=<SubBackward0>)
energies: tensor([-14.6296, -14.7648, -14.8001, -14.9442, -14.9974, -14.9460, -14.7476,
        -14.9249, -14.8598, -15.0175, -14.7536, -14.8725, -14.8003, -14.7614,
        -14.8097, -14.9403, -14.8552, -15.0631, -14.9184, -14.5561, -14.8963,
        -14.9621, -14.7222, -14